In [1]:
import re, json, pathlib, logging, time, argparse, pprint, warnings
import datetime as dt
import importlib.resources as rsrc
from itertools import chain
from operator import itemgetter 
#
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import nltk as nltk
from nltk.corpus import stopwords
from whoosh.lang.porter import stem
#from googletrans import Translator
from pygoogletranslation import Translator
import circlify as crcf
import matplotlib.pyplot as plt
import random

import polmap.polmap as plmp
import polmap.clean_docs as cldc
import postprocess.postprocess as pspr
from docx2python import docx2python
import docx2python.iterators as d2piter


##MM imports
#from polmap.polmap import make_directories, preprocess_text, doc2text, SDGrefs_mapper # replaced the keyword processing block

#import fromRtoPython

from docx2python import docx2python
 
#from polmap.polmap import make_directories, preprocess_text, doc2text # replaced the keyword processing block

In [2]:
mapping_test=pd.ExcelFile('mapping_test.xlsx')
mapping_test.sheet_names

count_df=mapping_test.parse('Count', index_col=0)
term_df=mapping_test.parse('Keywords', index_col=0)

In [3]:
count_df.head()

,Goal,Target,0,1,2,3,4,5,6,7,...,48,49,50,51,52,53,54,55,56,57
0,SDG 1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SDG 1,1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SDG 1,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SDG 1,1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SDG 1,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
mapping_df=pd.DataFrame()
for row1, row2 in zip(count_df.itertuples(), term_df.itertuples()):
    count=pd.DataFrame(data={'Count':row1,'Terms':row2}).iloc[3:]
    count['Goal'] = row1.Goal
    count['Target'] = row1.Target
    count = count.reindex(columns=['Goal','Target','Count','Terms'])   
    count.fillna(0, inplace=True)
    count = count[count.Count > 0]
    mapping_df = pd.concat([mapping_df, count])

In [15]:
mapping_df.head(100)

,Goal,Target,Count,Terms
9,SDG 2,2.1,1.0,food insecur
10,SDG 2,2.1,1.0,food all
19,SDG 2,2.2,1.0,healthi sustain diet
15,SDG 2,2.3,1.0,decent live farmer
7,SDG 2,2.4,2.0,organ farm
...,...,...,...,...
29,SDG 13,13.2,2.0,reduc emiss
30,SDG 13,13.2,1.0,climate-rel polici
32,SDG 13,13.2,1.0,integr climat chang risk manag practic
33,SDG 13,13.2,5.0,climat object
